In [1]:
%load_ext autoreload
%autoreload 2

from pangolin.interface import *
#from matplotlib import pyplot as plt
from pangolin.interface import *
from pangolin.loops import VMapRV
from pangolin import new_infer as infer

In [12]:
# express model in numpyro
import numpy as np
import numpyro
from numpyro import distributions as dist
import jax

N = 10 # num individuals
L = 3 # number of locations/alleles
K = 5 # number of groups

# M = 3
# K = 4
# N = 5
# V = 6
α = np.ones(K)
#β = np.ones(V)
def model():
    with numpyro.plate('N',N):
        # each individual's distribution over ancestral populations
        q = numpyro.sample('q',dist.Dirichlet(α))

    with numpyro.plate('K',K):
        with numpyro.plate('L',L):
            p = numpyro.sample('p',dist.Uniform(0,1))

    probs = q @ p.T

    g = numpyro.sample('g',dist.Binomial(2,probs))

    #with numpyro.plate('L'):
    #    with numpyro.plate('N'):


    #     φ = numpyro.sample('φ',dist.Dirichlet(β))
    # with numpyro.plate('M',M,dim=-2):
    #     θ = numpyro.sample('θ',dist.Dirichlet(α))
    #     with numpyro.plate('N',N):
    #         z = numpyro.sample('z',dist.Categorical(θ))
    #         w = numpyro.sample('w',dist.Categorical(φ[z,:]))

rng_key = jax.random.PRNGKey(0)
predictive = numpyro.infer.Predictive(model, num_samples=17)
samples = predictive(rng_key)

qs = samples['q']
ps = samples['p']
gs = samples['g']
# zs = samples['z']
# ws = samples['w']
#
print(f"{qs.shape=}")
print(f"{ps.shape=}")
print(f"{gs.shape=}")
# print(f"{φs.shape=}")
# print(f"{zs.shape=}")
# print(f"{ws.shape=}")


qs.shape=(17, 10, 5)
ps.shape=(17, 3, 5)
gs.shape=(17, 10, 3)
